In [1]:
from azureml.core import Workspace, Experiment

In [2]:
subscription_id = 'ef78944e-d555-4976-84de-e8954c8a9357'
resource_group = 'Machine_Learning'
workspace_name = 'Udemy_ML'

ws = Workspace(subscription_id, resource_group, workspace_name)

ws.write_config(path='.azureml')

exp = Experiment(workspace=ws, name='udacity-project1')

In [3]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

print('Experiment name: ' + exp.name)

Workspace name: Udemy_ML
Azure region: eastus2
Subscription id: ef78944e-d555-4976-84de-e8954c8a9357
Resource group: Machine_Learning
Experiment name: udacity-project1


In [4]:
run = exp.start_logging()      

In [5]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "project-1-CPU"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
from azureml.train.hyperdrive import choice, loguniform
import os
import shutil

# Specify parameter sampler

### YOUR CODE HERE ###

ps = RandomParameterSampling(
    {
    '--C': uniform(0.0, 1.0),
    '--max_iter': choice(10, 40, 80, 100, 120, 150)
    }
) 

# Specify a Policy

### YOUR CODE HERE ###

policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1) 


if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py

### YOUR CODE HERE ###

script_folder = './training'

shutil.copy('./train.py', script_folder)
est = SKLearn(source_directory =script_folder,
    entry_script = "train.py",
    compute_target=cpu_cluster_name)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.

### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig( 
    policy = policy,
    hyperparameter_sampling=ps,
    primary_metric_name = 'Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 12,
    max_concurrent_runs = 3,
    estimator = est
)


In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print (best_run.get_file_names())

['azureml-logs/55_azureml-execution-tvmps_7557a1d211bc6cb8317ded1b2b40782d6951f8a17419c140681c4981af79eb40_d.txt', 'azureml-logs/65_job_prep-tvmps_7557a1d211bc6cb8317ded1b2b40782d6951f8a17419c140681c4981af79eb40_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_7557a1d211bc6cb8317ded1b2b40782d6951f8a17419c140681c4981af79eb40_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/104_azureml.log', 'logs/azureml/dataprep/backgroundProcess.log', 'logs/azureml/dataprep/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/engine_spans_l_9ee4cba6-c8e0-4185-b57b-f7aee62f9f53.jsonl', 'logs/azureml/dataprep/python_span_l_9ee4cba6-c8e0-4185-b57b-f7aee62f9f53.jsonl', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']


In [9]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project1,HD_951dfee5-2b65-4418-abf8-ee11527f8493_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [10]:
model = best_run.register_model(model_name='hyperdrive_model', 
    model_path='./outputs/model.joblib')

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# TODO: Create TabularDataset using TabularDatasetFactory
# Data is located at:
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

file_path = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'
print(file_path)


https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv


In [12]:
ds = TabularDatasetFactory.from_delimited_files(file_path)

In [13]:
ds

{
  "source": [
    "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
  ],
  "definition": [
    "GetFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ]
}

In [14]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [15]:
import pandas as pd

x_train, x_test, y_train, y_test = train_test_split(x, y)
df_train = pd.concat([x_train, y_train], axis=1)

In [16]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=df_train,
    label_column_name='y',
    enable_onnx_compatible_models=True,
    n_cross_validations=5)

In [17]:
# Submit your automl run

### YOUR CODE HERE ###

exp = Experiment(workspace=ws, name='udacity-project1-autoML')

remote_run = exp.submit(automl_config, show_output=False)
RunDetails(remote_run).show()

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [18]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

from azureml.automl.runtime.onnx_convert import OnnxConverter
best_run_model, onnx_model= remote_run.get_output(return_onnx_model=True)
OnnxConverter.save_onnx_model(onnx_model, file_path="./automl_model.onnx")